# **MACHINE LEARNING NANODEGREE**
***
## **PROJETO FINAL**
***

## **Análise Preditiva de Cota para Exercício Parlamentar (CEAP)**

## ** PARTE 3 - Predição de Dados**
***

## ** DEPUTADO FRANCISCO CHAPADINHA**

***

## ** INTRODUÇÃO **

Realizaremos uma análise preditiva dos gastos para o ano de 2018 utilizando-se de algoritmo de aprendizado supervisionado.

É importante ressaltar que um deputado, salvo reeleição, terá registro de apenas quatros anos de despezas da CEAP. O que acaba sendo relativamente poucos dados os três primeiros anos para previsão dos gastos do último ano do mandato. Esta característica do dataset pode causar overfitting e ter impactos negativos com outliers.

Para contornar este problema devemos buscar utilizar algoritmos de menor complexidade, com menor número de parâmetros.
Tendo em vista estes desafios iremos utilizar dois tipos de lagoritmo:
> 1. LINEAR REGRESSION
> 2. LASSO REGRESSION
> 3. LARS LASSO REGRESSION
> 4. RIDGE REGRESSION
> 5. BAYESIAN RIDGE

Referência: https://goo.gl/gRYnWW

***

## **A. Importação de Biblioteca Python**

#### Serão utilizadas neste projeto as seguintes bibliotecas Python:

> 1. Pandas
> 2. Numpy
> 3. Scikit Learn
> 4. Seaborn


In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.metrics import r2_score
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import seaborn as sns

# pre-processing
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import LabelEncoder

# Cross validation
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import ShuffleSplit
from sklearn.model_selection import cross_val_predict

# Model selection
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import KFold

# Machine learning

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.grid_search import GridSearchCV

# Métrica
from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.metrics import make_scorer

# definindo que a execução do código na linha de baixo.
%matplotlib inline

# Ajustando globalmente o impedimento de notação científica
pd.options.display.float_format = '{:20,.2f}'.format

# Variável para definir a amostragem para head() e Tail()
H = 1

/Users/tec/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


***

## **B. Preparação de Dataset**


### B.1 Importando o dataset
Será utilizado o dataset individual criado para o **Dep. Domingos Neto**:

> - df_fco_con.csv.


In [2]:
display(os.listdir('dataset'))

['.DS_Store',
 'Ano-2015.csv',
 'Ano-2016.csv',
 'Ano-2017.csv',
 'df_dom_con.csv',
 'df_fco_con.csv',
 'df_roc_con.csv',
 'df_trienio_limpo.csv']

In [3]:
# importando o dataset limpo.
df = pd.read_csv('dataset/df_fco_con.csv', sep = ';', encoding='latin1')

In [4]:
df.Tipo.unique()

array(['Alimentação', 'C&L', 'Correios', 'Divulgação', 'Escritório',
       'Hospedagem', 'Passagens Aéreas', 'S/D', 'Taxi', 'Telefonia',
       'Consultorias'], dtype=object)

***
## C. DEPUTADO FRANCISCO CHAPADINHA

## C.1 CRIANDO X e y

In [5]:
# Criação da variáveis X e y.
X = df
y = X.pop('Valor')

In [6]:
X.head(H)

,Tipo,Ano
0,Alimentação,2015


In [7]:
y.head(H)

0               5,419.78
Name: Valor, dtype: float64

In [8]:
X.shape

(31, 2)

## C.2 CRIANDO UM DATAFRAME PARA PREDIÇÃO DE 2018

Antes de aplicarmos get_dummies na feature Tipo, precisamos listar todo o conteúdo único contido para criarmos o dataframe de predição para 2018.

In [9]:
uniq = X.Tipo.unique()
#display(uniq)
uniq = pd.DataFrame(uniq, columns=['Tipo'])
#display(uniq)
df_2018 = pd.get_dummies(uniq, columns = ['Tipo'])
#display(df_tipo)
df_2018['Ano'] = 2018
#ano = df_2018.pop('Ano')
#df_2018['Ano'] = ano
df_2018.head(12)
#ano = df_2018.pop('Ano')
df_2018 = df_2018.drop('Ano', axis = 1)
df_2018.insert(0, 'Ano', 2018)
df_2018.head(20)

,Ano,Tipo_Alimentação,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Taxi,Tipo_Telefonia
0,2018,1,0,0,0,0,0,0,0,0,0,0
1,2018,0,1,0,0,0,0,0,0,0,0,0
2,2018,0,0,0,1,0,0,0,0,0,0,0
3,2018,0,0,0,0,1,0,0,0,0,0,0
4,2018,0,0,0,0,0,1,0,0,0,0,0
5,2018,0,0,0,0,0,0,1,0,0,0,0
6,2018,0,0,0,0,0,0,0,1,0,0,0
7,2018,0,0,0,0,0,0,0,0,1,0,0
8,2018,0,0,0,0,0,0,0,0,0,1,0
9,2018,0,0,0,0,0,0,0,0,0,0,1


## C.3 TRATANDO FEATURE COM CLASSIFICAÇÃO
Tendo em vista que os algoritmos de machine learning da biblioteca Scikit Learn não tratam automáticamente com colunas com tipo de dados Objeto utilizaremos a função get_dummies do Pandas.
Esta função irá criar colunas adicionais para representar cada tipo de gasto existente e indicará através de 0 e 1 se ela está presente na linha ou não.
Maiores informações podem ser encontradas neste link: 
> - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

In [10]:
# Vamos utilizar get_dummies para tratar os dados não numéricos da feature 'Tipo'.
X = pd.get_dummies(X, columns = ['Tipo'])

In [11]:
X.head(H)

,Ano,Tipo_Alimentação,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Taxi,Tipo_Telefonia
0,2015,1,0,0,0,0,0,0,0,0,0,0


In [12]:
X.shape

(31, 12)

## C.4 'LOGARITMIZAÇÃO'

In [13]:
# Aplicação de logaritmica
    
yl = np.log(y)
Xl = X

## C.5 DATA SPLIT

Como estaremos realizando o uso dos dados de 2015, 2016 e 2017 para realizar a predição dos gastos de 2018 de três deputados, iremos então separar os dados de 2015 e 2016 para serem utilizados como train e os dados de 2017 para test.
O splot de dados será realizado COM e SEM a aplicação de log em y.

In [14]:
# Test split percentual e random state.
per = 0.20
random = 7

# data split SEM LOG.
# Data split com 20% de dados para treinamento.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = per, random_state = random)

# data split COM LOG.
# Data split com 20% de dados para treinamento.
Xl_train, Xl_test, yl_train, yl_test = train_test_split(Xl, yl, test_size = per, random_state = random)

In [15]:
X_train.head(H)

,Ano,Tipo_Alimentação,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Taxi,Tipo_Telefonia
30,2017,0,0,0,0,0,0,0,0,0,0,1


In [16]:
X_test.head(H)

,Ano,Tipo_Alimentação,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Taxi,Tipo_Telefonia
2,2015,0,0,0,1,0,0,0,0,0,0,0


In [17]:
y_train.head(H)

30               4,955.71
Name: Valor, dtype: float64

In [18]:
yl_train.head(H)

30                   8.51
Name: Valor, dtype: float64

In [19]:
y_test.head(H)

2               3,926.35
Name: Valor, dtype: float64

In [20]:
yl_test.head(H)

2                   8.28
Name: Valor, dtype: float64

## C.6 PREDIÇÃO POR REGRESSÃO LINEAR

### CRIAÇÃO DE FUNÇÃO PARA CONSTRUÇÃO DE TABELA RESULTADO


In [21]:
# Criando tabela com o resultado da regressão
def tabela_resultado(modelo, df):
    pr = modelo.predict(df)
    pr = pd.DataFrame(data=pr)
    #display(pr)
    pr['Valor'] = pr
    #display(pr)
    pr = pr.pop('Valor')
    #display(pr)
    co = df_2018.columns
    co = pd.DataFrame(data=co)
    #display(co)
    co.drop(0, axis = 0, inplace = True)
    co['Tipo'] = co
    co = co.pop('Tipo')
    co = co.reset_index(drop=True)
    #display(co)
    results = pd.concat([co, pr], axis = 1)
    return results
    #display(results)

### CRIAÇÃO DE DATAFRAME PARA PREDIÇÃO PARA 2018

***
## D. LINEAR REGRESSION

### D.1 SEM APLICAÇÃO DE  LOG

In [22]:
lr = linear_model.LinearRegression()
lr.fit(X_train,y_train)
display('Score para train ', lr.score(X_train,y_train).round(4))
display('Score para test ', lr.score(X_test,y_test).round(4))


'Score para train '

0.9031

'Score para test '

0.7079

### D.2 PREDIÇÃO PARA 2018 SEM LOG

In [23]:
# Predição
# Data recover
resultado = tabela_resultado(lr, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"-9,444.10"
1,Tipo_C&L,"55,966.35"
2,Tipo_Consultorias,"-2,342.63"
3,Tipo_Correios,"380,133.69"
4,Tipo_Divulgação,"12,465.71"
5,Tipo_Escritório,"-3,632.99"
6,Tipo_Hospedagem,"46,951.97"
7,Tipo_Passagens Aéreas,"133,110.36"
8,Tipo_S/D,"-14,777.30"
9,Tipo_Taxi,"-3,872.93"


619828.4

***
### D.3 COM APLICAÇÃO DE  LOG

In [24]:
lrl = linear_model.LinearRegression()
lrl.fit(Xl_train,yl_train)
display('Score para train ', lrl.score(Xl_train,yl_train).round(4))
display('Score para test ', lrl.score(Xl_test,yl_test).round(4))


'Score para train '

0.8939

'Score para test '

0.6094

### D.4 PREDIÇÃO PARA 2018 COM LOG

In [25]:
# Predição
#tabela_resultado(lrl, df_2018)
# Data recover
resultado = tabela_resultado(lrl, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,372.36
1,Tipo_C&L,"35,903.25"
2,Tipo_Consultorias,"2,836.55"
3,Tipo_Correios,"128,487.67"
4,Tipo_Divulgação,"3,034.86"
5,Tipo_Escritório,"1,343.69"
6,Tipo_Hospedagem,"15,721.68"
7,Tipo_Passagens Aéreas,"37,406.96"
8,Tipo_S/D,5.57
9,Tipo_Taxi,"2,212.32"


238810.53

***

***
## E. LASSO REGRESSION

***

### E.1 SEM APLICAÇÃO DE LOG

In [26]:
from sklearn import linear_model
la = linear_model.Lasso(alpha = 100)
la.fit(X_train,y_train)
display('Score para train ', la.score(X_train,y_train).round(4))
display('Score para test ', la.score(X_test,y_test).round(4))

'Score para train '

0.903

'Score para test '

0.7142

### E.2 PREDIÇÃO PARA 2018  SEM LOG

In [27]:
resultado = tabela_resultado(la, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"-8,214.95"
1,Tipo_C&L,"53,771.65"
2,Tipo_Consultorias,-819.40
3,Tipo_Correios,"379,773.16"
4,Tipo_Divulgação,"12,991.07"
5,Tipo_Escritório,"-2,103.72"
6,Tipo_Hospedagem,"46,308.69"
7,Tipo_Passagens Aéreas,"132,356.66"
8,Tipo_S/D,"-13,016.31"
9,Tipo_Taxi,"-2,335.02"


623102.73

***

### E.3 COM APLICAÇÃO DE LOG

In [28]:
from sklearn import linear_model
lal = linear_model.Lasso(alpha = 100)
lal.fit(Xl_train,yl_train)
display('Score para train ', lal.score(Xl_train,yl_train).round(4))
display('Score para test ', lal.score(Xl_test,yl_test).round(4))

'Score para train '

0.0

'Score para test '

-0.326

### E.4 PREDIÇÃO PARA 2018 COM LOG

In [29]:
resultado = tabela_resultado(lal, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"9,702.21"
1,Tipo_C&L,"9,702.21"
2,Tipo_Consultorias,"9,702.21"
3,Tipo_Correios,"9,702.21"
4,Tipo_Divulgação,"9,702.21"
5,Tipo_Escritório,"9,702.21"
6,Tipo_Hospedagem,"9,702.21"
7,Tipo_Passagens Aéreas,"9,702.21"
8,Tipo_S/D,"9,702.21"
9,Tipo_Taxi,"9,702.21"


106724.27

***

***

## F. LARS LASSO

### F.1 SEM APLICAÇÃO DE LOG

In [30]:
from sklearn import linear_model
ll = linear_model.LassoLars(alpha=.1)
ll.fit(X_train,y_train)
display('Score para train ', ll.score(X_train,y_train).round(4))
display('Score para test ', ll.score(X_test,y_test).round(4))

'Score para train '

0.9031

'Score para test '

0.7079

### F.2 PREDIÇÃO PARA 2018 SEM LOG

In [31]:
resultado = tabela_resultado(ll, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"-9,441.30"
1,Tipo_C&L,"55,964.75"
2,Tipo_Consultorias,"-2,339.88"
3,Tipo_Correios,"380,133.90"
4,Tipo_Divulgação,"12,466.91"
5,Tipo_Escritório,"-3,630.24"
6,Tipo_Hospedagem,"46,952.23"
7,Tipo_Passagens Aéreas,"133,110.24"
8,Tipo_S/D,"-14,773.79"
9,Tipo_Taxi,"-3,870.18"


619842.43

### F.3 COM APLICAÇÃO DE LOG

In [32]:
from sklearn import linear_model
lll = linear_model.LassoLars(alpha=.1)
lll.fit(Xl_train,yl_train)
display('Score para train ', lll.score(Xl_train,yl_train).round(4))
display('Score para test ', lll.score(Xl_test,yl_test).round(4))

'Score para train '

0.6747

'Score para test '

-0.0223

### F.4 PREDIÇÃO PARA 2018 COM LOG

In [33]:
resultado = tabela_resultado(lll, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"4,043.79"
1,Tipo_C&L,"11,834.41"
2,Tipo_Consultorias,"11,834.41"
3,Tipo_Correios,"104,441.04"
4,Tipo_Divulgação,"11,834.41"
5,Tipo_Escritório,"11,834.41"
6,Tipo_Hospedagem,"12,078.24"
7,Tipo_Passagens Aéreas,"21,889.06"
8,Tipo_S/D,110.37
9,Tipo_Taxi,"11,834.41"


213568.98

***

***

## G. RIDGE REGRESSION

### G.1 SEM APLICAÇÃO DE LOG

In [34]:
from sklearn import linear_model
ri = linear_model.Ridge (alpha = .5)
ri.fit(X_train,y_train)
display('Score para train ', ri.score(X_train,y_train).round(4))
display('Score para test ', ri.score(X_test,y_test).round(4))

'Score para train '

0.8825

'Score para test '

0.6287

### G.2 PREDIÇÃO PARA 2018 SEM LOG

In [35]:
resultado = tabela_resultado(ri, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"-3,711.46"
1,Tipo_C&L,"54,057.67"
2,Tipo_Consultorias,"6,609.48"
3,Tipo_Correios,"330,212.36"
4,Tipo_Divulgação,"15,068.38"
5,Tipo_Escritório,"5,577.20"
6,Tipo_Hospedagem,"44,480.69"
7,Tipo_Passagens Aéreas,"114,189.64"
8,Tipo_S/D,"-4,902.73"
9,Tipo_Taxi,"5,385.24"


595666.27

### G.3 COM APLICAÇÃO DE LOG

In [36]:
from sklearn import linear_model
ril = linear_model.Ridge (alpha = .5)
ril.fit(Xl_train,yl_train)
display('Score para train ', ril.score(Xl_train,yl_train).round(4))
display('Score para test ', ril.score(Xl_test,yl_test).round(4))

'Score para train '

0.8604

'Score para test '

0.7284

### G.4 PREDIÇÃO PARA 2018 COM LOG

In [37]:
resultado = tabela_resultado(ril, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,701.56
1,Tipo_C&L,"20,340.90"
2,Tipo_Consultorias,"3,799.70"
3,Tipo_Correios,"105,053.35"
4,Tipo_Divulgação,"4,237.17"
5,Tipo_Escritório,"2,090.05"
6,Tipo_Hospedagem,"16,953.72"
7,Tipo_Passagens Aéreas,"31,853.23"
8,Tipo_S/D,29.44
9,Tipo_Taxi,"3,114.55"


199805.31

***

***

## H. BAYESIAN RIDGE

### H.1 SEM APLICAÇÃO DE LOG

In [38]:
br = linear_model.BayesianRidge()
br.fit(X_train,y_train)
display('Score para train ', br.score(X_train,y_train).round(4))
display('Score para test ', br.score(X_test,y_test).round(4))

'Score para train '

0.0

'Score para test '

-0.5611

### H.2 PREDIÇÃO PARA 2018 SEM LOG

In [39]:
resultado = tabela_resultado(br, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"76,384.90"
1,Tipo_C&L,"76,384.90"
2,Tipo_Consultorias,"76,384.90"
3,Tipo_Correios,"76,384.90"
4,Tipo_Divulgação,"76,384.90"
5,Tipo_Escritório,"76,384.90"
6,Tipo_Hospedagem,"76,384.90"
7,Tipo_Passagens Aéreas,"76,384.90"
8,Tipo_S/D,"76,384.90"
9,Tipo_Taxi,"76,384.90"


840233.92

***

### H.3 COM APLICAÇÃO DE LOG

In [40]:
brl = linear_model.BayesianRidge()
brl.fit(Xl_train,yl_train)
display('Score para train ', brl.score(Xl_train,yl_train).round(4))
display('Score para test ', brl.score(Xl_test,yl_test).round(4))

'Score para train '

0.8838

'Score para test '

0.7204

### H.4 PREDIÇÃO PARA 2018 COM LOG

In [41]:
resultado = tabela_resultado(brl, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,528.97
1,Tipo_C&L,"25,464.38"
2,Tipo_Consultorias,"3,348.29"
3,Tipo_Correios,"117,758.90"
4,Tipo_Divulgação,"3,683.60"
5,Tipo_Escritório,"1,719.96"
6,Tipo_Hospedagem,"16,714.11"
7,Tipo_Passagens Aéreas,"34,763.26"
8,Tipo_S/D,14.02
9,Tipo_Taxi,"2,682.78"


218328.39

Referências:
> 1. https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/
> 2.  https://chrisalbon.com/machine_learning/preprocessing_structured_data/preprocessing_categorical_features/
> 3. https://www.packtpub.com/mapt/book/big_data_and_business_intelligence/9781787286382/2/ch02lvl1sec23/working-with-categorical-variables
> 4. http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
> 5. http://scikit-learn.org/stable/auto_examples/plot_cv_predict.html#sphx-glr-auto-examples-plot-cv-predict-py
> 6. http://fastml.com/how-to-use-pd-dot-get-dummies-with-the-test-set/
> 7. Scikit-learn Cookbook - Second Edition - Julian Avila, Trent Hauck - November 2017 - https://www.packtpub.com/mapt/book/all_books/9781787286382
> 8. scikit-learn : Machine Learning Simplified - Raúl Garreta, Guillermo Moncecchi, Trent Hauck, Gavin Hackeling - November 2017 - https://www.packtpub.com/mapt/book/all_books/9781788833479
> 9. Mastering Machine Learning with scikit-learn - Second Edition - Gavin Hackeling - July 2017 - https://www.packtpub.com/mapt/book/all_books/9781788299879
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 